In [1]:
import torch
from torch import nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import stack
from fetalnav.models.spn_models import vgg11_sp, vgg13_sp

model = vgg11_sp(num_classes=7, num_maps=512, batch_norm=True, in_channels=1)


In [2]:
model

SPNetWSL(
  (features): Sequential(
    (0): Conv2d (1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (4): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (8): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (10): ReLU(inplace)
    (11): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (15): Conv2d (256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16): Ba

In [3]:
modelcuda = torch.nn.DataParallel(model).cuda()
input_batch = torch.randn(4, 1, 112, 112)
o = modelcuda(Variable(input_batch))
print(o.shape)
o

torch.Size([4, 7])


Variable containing:
 0.1057 -0.0844 -0.0116  0.1061 -0.1062  0.0735 -0.0811
 0.0819 -0.0806  0.0044  0.1203 -0.0929  0.0458 -0.0578
 0.1003 -0.0786 -0.0178  0.1076 -0.0700  0.0541 -0.0741
 0.1177 -0.1038  0.0020  0.0961 -0.0768  0.0466 -0.0603
[torch.cuda.FloatTensor of size 4x7 (GPU 0)]

In [10]:
o[:,:,1]

Variable containing:
 0.2292 -0.1126  0.0753  0.0207 -0.0055  0.0044  0.0347
 0.2315 -0.1079  0.0622  0.0321  0.0184 -0.0009  0.0171
 0.2193 -0.1165  0.0771  0.0354  0.0232  0.0244  0.0029
 0.2274 -0.0976  0.0724  0.0145  0.0048 -0.0054  0.0242
[torch.cuda.FloatTensor of size 4x7 (GPU 0)]

In [8]:
out = 0.
t


    1     1     0     0     0     0     0
    0     1     1     1     1     0     0
    0     0     1     0     0     0     0
    0     0     0     1     0     0     0
[torch.FloatTensor of size 4x7]

In [4]:

class MultiLinearMultiLabelSoftMarginLoss(nn.MultiLabelSoftMarginLoss):

    def forward(self, input, target):
        out = 0.
        for idx in range(input.size(1)):
            class_input  = input[:, idx]
            class_target = target[:, idx]
            v = F.multilabel_soft_margin_loss(class_input, class_target, weight=self.weight)
            out += v
        return out

t = torch.zeros(o.size(0), o.size(1))
t[0,0] = t[0,1] = 1.
t[1,1] = t[1,2] = 1.
t[2,2] = t[1,3] = 1.
t[3,3] = t[1,4] = 1.
state = {}
state['output']= o.data.cpu()
state['target']= t


loss = MultiLinearMultiLabelSoftMarginLoss()

loss(Variable(state['output']), Variable(state['target']))

IndexError: trying to index 3 dimensions of a 2 dimensional tensor

In [2]:
import numpy as np
import pandas as pd

from IPython.display import display, HTML

In [19]:

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from scipy.misc import imresize
from scipy.ndimage import label
from spn.modules import SoftProposal

# helper functions

def hook_spn(model):
    if not (hasattr(model, 'sp_hook') and hasattr(model, 'fc_hook')):
        model._training = model.training
        model.train(False)
        
        def _pre_sp_hook(self, input, output):
            print(self.name)
            self.parent_modules[0].class_response_maps = output
        def sp_hook(self, input, output):
            print(self.name)
            self.parent_modules[0].class_response_maps = output
        def _fc_hook(self, input, output):
            if hasattr(self.parent_modules[0], 'class_response_maps'):
                self.parent_modules[0].class_response_maps = F.conv2d(self.parent_modules[0].class_response_maps, self.weight.unsqueeze(-1).unsqueeze(-1))
            else:
                raise RuntimeError('The SPN is broken, please recreate it.')
                
        sp_layers = []
        fc_layers = []
        for mod in model.modules():
            if isinstance(mod, SoftProposal):
                sp_layers.append(mod)
            elif isinstance(mod, torch.nn.Linear):
                fc_layers.append(mod)
        
        if not len(sp_layers) or not len(sp_layers):
            raise RuntimeError('Invalid SPN model')
        else:
            for s in sp_layers:
                s.parent_modules = [model]
                model.sp_hook = s.register_forward_hook(_sp_hook)
            for s in sp_layers:
                s.parent_modules = [model]
                model.fc_hook = s.register_forward_hook(_fc_hook)
    return model

def generate_outputs(model, in_var):

    from spn import hook_spn
    from torch.nn import functional as F

    if in_var.ndimension() == 3:
        input = in_var.unsqueeze(0)
    assert in_var.size(0) == 1, 'Batch processing is currently not supported'
    # enable spn inference mode
    model = hook_spn(model)
    # predict scores
    scores = torch.nn.Softmax(dim=1)(model(in_var)).data.cpu().squeeze()
    # instantiate maps
    maps = F.upsample(model.class_response_maps, size=(in_var.size(2), in_var.size(3)), mode='bilinear').data
    return scores.numpy(), maps

input_batch = torch.randn(1, 1, 112, 112)
a = generate_outputs(model, Variable(input_batch.cuda()))

a[1].shape

torch.Size([1, 2, 112, 112])

In [23]:
a = nn.Linear(10,2)
a.named_parameters()

<generator object Module.named_parameters at 0x7fc50160c678>